## Dependencies

In [5]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np


import torch
from torch.utils.data import DataLoader,random_split
from torchvision import transforms


import importlib
import dataloader
import config
import model
import train


importlib.reload(dataloader)
importlib.reload(config)
importlib.reload(model)
importlib.reload(train)

from train import train,validate,inference
from dataloader import create_dataloaders
from config import Config
from model import ModelV1,initialize_weights


config = Config()

## Prepare Dataset

In [6]:
train_loader, val_loader, hb_mean, hb_std = create_dataloaders(
    data_dir=config.data_dir,
    batch_size=config.batch_size,
    test_split=config.test_split,
)

## Create Model

In [7]:
model_obj = ModelV1(freeze_backbone=True, unfreeze_from_layer='layer4')
# if config.load_model:
#     model_obj.load_state_dict(torch.load(config.model_path))
# else:
#     initialize_weights(model_obj)

## Train

In [8]:
train(model = model_obj,dataloader = train_loader,val_loader = val_loader, config = config)

Training on Device: mps

[Train] Epoch [1/20], Batch [208/208], Loss: 0.2413  ------> Epoch [1/20] Completed. Avg Loss: 1.0243
[Validation] Batch [23/23] - Batch Loss: 0.3471  ------> Validation Loss: 1.0909, R2: 0.0031, MAPE: 115.0462%
[Train] Epoch [2/20], Batch [208/208], Loss: 0.7974  ------> Epoch [2/20] Completed. Avg Loss: 0.9172
[Validation] Batch [23/23] - Batch Loss: 0.3146  ------> Validation Loss: 1.0819, R2: 0.0113, MAPE: 103.7414%
[Train] Epoch [3/20], Batch [208/208], Loss: 0.4865  ------> Epoch [3/20] Completed. Avg Loss: 0.9101
[Validation] Batch [23/23] - Batch Loss: 0.3304  ------> Validation Loss: 1.1011, R2: -0.0063, MAPE: 102.3589%
[Train] Epoch [4/20], Batch [208/208], Loss: 0.0732  ------> Epoch [4/20] Completed. Avg Loss: 0.9018
[Validation] Batch [23/23] - Batch Loss: 0.2376  ------> Validation Loss: 1.0450, R2: 0.0449, MAPE: 127.7721%
[Train] Epoch [5/20], Batch [208/208], Loss: 1.2320  ------> Epoch [5/20] Completed. Avg Loss: 0.7742
[Validation] Batch [23/2

KeyboardInterrupt: 

## Validate Model

In [11]:
_ = validate(model = model_obj, dataloader = val_loader, config = Config())

[Validation] Batch [23/23] - Batch Loss: 0.1085
[Validation] Completed. Avg Loss: 1.2091

[Validation Metrics]
MSE: 1.2091
RMSE: 1.0996
MAE: 0.7630
R2: -0.1050
MAPE (%): 155.6261


## inference

In [ ]:
for i in range(1,10):
    paths = f"/Users/maheshsaravanan/Documents/HemoScan/Dataset/{i:04d}"
    inference(model = model_obj, path = paths, mean = dataset.hb_mean, std = dataset.hb_std)